In [58]:
import Pkg
Pkg.activate("../../../FinancialPlanner")

  Activating project at `c:\Users\matsz\programowanie\Optymalizacja_portfela\FinancialPlanner`


In [59]:
using Revise
using LinearAlgebra
using Distributions
using Random
using StatsPlots
using PDMats
using StatsBase
using Base.Iterators
using DataFrames, TimeSeries, XLSX
using PrettyTables

include("../VARs/utils.jl")
includet("../bootstrap_model.jl")

In [60]:
df = DataFrame(XLSX.readtable("../../data/usa_data.xlsx", "Data", infer_eltypes=true))
data_source = collapse(TimeArray(df; timestamp = :Date), Dates.month, last)

freq = 12
returns = percentchange(data_source[:Total_Stock, :Price_10Y], :log)
cpi = percentchange(data_source[:CPI], :log)
real_returns = returns .- cpi
start = Date(1955, 01,01)
end_d = Date(2024, 12, 31)


display("Returns")
display(returns_summarystats(to(from(returns, start),end_d), freq))
display("Real_returns")
display(returns_summarystats(to(from(real_returns, start),end_d), freq))


display("Correlations")
display(cor(values(to(from(real_returns, start),end_d)))) 

"Returns"

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock,0.1473,1.8543,0.1012,0.4365,-2.9008,-0.6469,-0.1883,0.0258,2.1941,0.6874,0.1474
Price_10Y,0.0755,1.3969,0.0531,0.1909,-0.9866,0.3097,-0.0893,0.1022,1.9867,0.7037,0.0377


nothing

"Real_returns"

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock_CPI,0.1479,1.7607,0.0658,0.4053,-2.9112,-0.596,-0.2186,0.032,1.9119,0.445,0.1107
Price_10Y_CPI,0.078,1.279,0.0177,0.1544,-1.1677,0.2268,-0.1322,0.1437,2.0153,0.2268,0.0058


nothing

"Correlations"

2×2 Matrix{Float64}:
 1.0        0.0980477
 0.0980477  1.0

In [61]:
assets_names = [:Total_Stock_CPI, :Price_10Y_CPI]
data = to(from(real_returns[assets_names], start),end_d) 

assets_names = [:Total_Stock_CPI, :Price_10Y_CPI]
data = to(from(real_returns[assets_names], start),end_d) 

ret_mean = mean(values(data), dims=1)[1,:]
ret_cov = cov(values(data), dims=1)

T = 25 * freq
n_samples = 10_000
n_assets = length(assets_names)

scenarios = zeros(n_assets, T, n_samples)

for t in 1:T 
    scenarios[:,t, :] = rand(MvNormal(ret_mean, ret_cov), n_samples)
end

periods = [1,5, 10, 25]
ret_in_years = cum_returns_in_periods(scenarios, periods, freq, true)
print_scenarios_summary(ret_in_years, assets_names, string.(periods))
n_assets = length(assets_names)
for a in 1:n_assets
    print_scenarios_percentiles(ret_in_years[a, :, :], [.01, .025, .05, .25, .5, .75, .95, .975, .99], string.(periods), string.(assets_names[a]))
end  


,Total_Stock_CPI,Price_10Y_CPI
1,0.0654,0.0174
5,0.066,0.0175
10,0.0657,0.0174
25,0.0659,0.0175


,Total_Stock_CPI,Price_10Y_CPI
1,0.1482,0.0783
5,0.0667,0.0348
10,0.0466,0.0245
25,0.0298,0.0156


,Total_Stock_CPI,Price_10Y_CPI
1,0.0127,-0.0071
5,-0.0395,0.0222
10,-0.0211,0.0272
25,-0.0432,-0.0045


,Total_Stock_CPI,Price_10Y_CPI
1,0.0431,-0.044
5,0.008,0.0098
10,-0.0441,0.0541
25,0.0259,0.0289


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.2764,-0.2269,-0.1792,-0.0342,0.066,0.1646,0.3093,0.3543,0.4186
5,-0.0894,-0.0665,-0.0447,0.0223,0.066,0.1111,0.1753,0.1955,0.219
10,-0.0435,-0.0251,-0.0113,0.034,0.0658,0.097,0.143,0.1562,0.1709
25,-0.0054,0.0063,0.0165,0.0465,0.0661,0.0861,0.114,0.1242,0.1346


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.1622,-0.1365,-0.1111,-0.0366,0.0179,0.0705,0.1476,0.1707,0.1974
5,-0.0634,-0.0504,-0.0395,-0.0058,0.0175,0.0408,0.075,0.0869,0.099
10,-0.0394,-0.0297,-0.0229,0.0006,0.0174,0.0336,0.0579,0.0662,0.0742
25,-0.0187,-0.0131,-0.008,0.0071,0.0174,0.0281,0.0434,0.0486,0.0535


In [62]:
mdd, mddl = max_drawdown_and_length(scenarios[1,:,:])

display(assets_names[1])
display("mdd")
display(quantile(mdd, [.03, .25, .5, .75, .97])')
display("mddl")
display(quantile(mddl, [.03, .25, .5, .75, .97])')

mdd, mddl = max_drawdown_and_length(scenarios[2,:,:])

display(assets_names[2])
display("mdd")
display(quantile(mdd, [.03, .25, .5, .75, .97])')
display("mddl")
display(quantile(mddl, [.03, .25, .5, .75, .97])')

:Total_Stock_CPI

"mdd"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 0.214692  0.291869  0.347904  0.420833  0.58433

"mddl"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 29.0  51.0  72.0  104.0  206.03

:Price_10Y_CPI

"mdd"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 0.14349  0.205382  0.255803  0.319092  0.46791

"mddl"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 41.0  76.0  113.0  168.0  285.0